In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-07-17T13:01:57.130906-04:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.10.0

Compiler    : GCC 12.3.0
OS          : Linux
Release     : 5.15.0-76-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 16
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: 
  GPU 0: Quadro RTX 5000



In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import sklearn
import gc
import os

In [5]:
%watermark --iversions

numpy  : 1.25.1
pandas : 2.0.3
sklearn: 1.3.0



In [6]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [7]:
train = pd.read_csv('../input/train.csv.zip')
test = pd.read_csv('../input/test.csv.zip')
sample_submission = pd.read_csv('../input/sample_submission.csv.zip')

In [8]:
columns = test.columns[1:]

In [9]:
X = train[columns].values
X_test = test[columns].values
Y = train.target.values

In [10]:
%%time
model = HistGradientBoostingClassifier()
model.fit(X, Y)
train_pred = model.predict_proba(X)[:,1]
test_pred = model.predict_proba(X_test)[:,1]

CPU times: user 26.2 s, sys: 35.2 ms, total: 26.3 s
Wall time: 3.98 s


In [11]:
train_pred

array([0.0545994 , 0.02885506, 0.02146885, ..., 0.0154268 , 0.0234167 ,
       0.01791986])

In [12]:
roc_auc_score(Y, train_pred)

0.6830964281189357

In [13]:
gini_normalized(Y, train_pred)

0.3661928567201128

In [14]:
sample_submission['target'] = test_pred

In [15]:
sample_submission.to_csv('../submissions/HGBC_0.csv', index=False)

On the leaderbaord this submission scores 0.27123 on the public test set, and 0.27399 on the private dataset.